In [184]:
%%time
train = pd.read_csv('./input/train.csv')

CPU times: user 5.91 ms, sys: 4 ms, total: 9.91 ms
Wall time: 9.17 ms


## 特征预处理

In [185]:
# 缺失值填充，离散型变量转换成数值


def feature_preprocessing(data):
    data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'male' else 0)
    data['Embarked'] = data['Embarked'].fillna(
        data['Embarked'].mode().values[0])
    data['Age'] = data['Age'].fillna(data['Age'].median())

    Pclass_dummies = pd.get_dummies(data['Pclass'], prefix='Pclass_')
    Embarked_dummies = pd.get_dummies(data['Embarked'], prefix='Embarked')

    # 丢弃姓名、客舱 Cabin 、船票信息 Ticket
    data.drop(
        columns=[
            'Name', 'Cabin', 'Ticket', 'Pclass', 'Embarked', 'PassengerId'
        ],
        inplace=True)

    data = pd.concat([data, Pclass_dummies, Embarked_dummies], axis=1)
    return data


train = feature_preprocessing(train)

In [186]:
train.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Pclass__1,Pclass__2,Pclass__3,Embarked_C,Embarked_Q,Embarked_S
0,0,1,22.0,1,0,7.2500,0,0,1,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0,1,0,0
2,1,0,26.0,0,0,7.9250,0,0,1,0,0,1
3,1,0,35.0,1,0,53.1000,1,0,0,0,0,1
4,0,1,35.0,0,0,8.0500,0,0,1,0,0,1


In [187]:
def split_X_y(data):

    X = data.drop(['Survived'], axis=1)
    y = data['Survived']
    return X, y

X, y = split_X_y(train)

In [188]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=666)

## 使用不同的模型



In [189]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

dt = DecisionTreeClassifier(random_state=666)
lr = LogisticRegression()
rfc = RandomForestClassifier(random_state=666)
svc = SVC(kernel='rbf')
ada = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
xgb = XGBClassifier()

models = [('DecisionTreeClassifier', dt), ('LogisticRegression', lr),
          ('RandomForestClassifier', rfc), ('svc', svc),
          ('AdaBoostClassifier', ada), ('GradientBoostingClassifier',
                                        gbc), ('XBClassifier', xgb)]

In [190]:
from sklearn.model_selection import cross_val_score


def predict_by_different_models(models):
    res = dict()
    for model in models:
        model_name = model[0]
        clf = model[1]
        clf.fit(X_train, y_train)
        train_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)
        res[model_name] = {
            'train_score': train_score,
            'test_score': test_score
        }
    return res

In [191]:
%%time

res = predict_by_different_models(models)

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


CPU times: user 258 ms, sys: 13 ms, total: 271 ms
Wall time: 273 ms


In [202]:
res = pd.DataFrame(res).T
res.reindex(columns=['train_score', 'test_score'])

,train_score,test_score
test_score,NaN,NaN
train_score,NaN,NaN


可以看出，随机森林的效果最好。
把全部训练数据用上。

## 使用网格搜索选择超参数

### 决策树

In [196]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

entropy_thresholds = np.linspace(0, 1, 50)
gini_thresholds = np.linspace(0, 0.5, 50)

parameters = [{
    'criterion': ['entropy'],
    'min_impurity_split': entropy_thresholds
}, {
    'criterion': ['gini'],
    'min_impurity_split': gini_thresholds
}, {
    'max_depth': range(2, 10)
}, {
    'min_samples_split': range(2, 30, 2)
}]

dt = DecisionTreeClassifier()
clf = GridSearchCV(dt, parameters, cv=5, verbose=2)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 122 candidates, totalling 610 fits
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.0 .......................
[CV] ........ criterion=entropy, min_impurity_split=0.0, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.02040816326530612 .......
[CV]  criterion=entropy, min_impurity_split=0.02040816326530612, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.02040816326530612 .......
[CV]  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value

[CV]  criterion=entropy, min_impurity_split=0.12244897959183673, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.12244897959183673 .......
[CV]  criterion=entropy, min_impurity_split=0.12244897959183673, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.12244897959183673 .......
[CV]  criterion=entropy, min_impurity_split=0.12244897959183673, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.12244897959183673 .......
[CV]  criterion=entropy, min_impurity_split=0.12244897959183673, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.14285714285714285 .......
[CV]  criterion=entropy, min_impurity_split=0.14285714285714285, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.14285714285714285 .......
[CV]  criterion=entropy, min_impurity_split=0.14285714285714285, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.14285714285714285 .......
[CV]  criterion=entropy, min_impurity_split=0.14285714285714285, total=   0.0s
[CV] criterion=entrop

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/

[CV]  criterion=entropy, min_impurity_split=0.24489795918367346, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.24489795918367346 .......
[CV]  criterion=entropy, min_impurity_split=0.24489795918367346, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.26530612244897955 .......
[CV]  criterion=entropy, min_impurity_split=0.26530612244897955, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.26530612244897955 .......
[CV]  criterion=entropy, min_impurity_split=0.26530612244897955, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.26530612244897955 .......
[CV]  criterion=entropy, min_impurity_split=0.26530612244897955, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.26530612244897955 .......
[CV]  criterion=entropy, min_impurity_split=0.26530612244897955, total=   0.0s
[CV] criterion=entropy, min_impurity_split=0.26530612244897955 .......
[CV]  criterion=entropy, min_impurity_split=0.26530612244897955, total=   0.0s
[CV] criterion=entrop

...

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/liwei/

[CV]  criterion=gini, min_impurity_split=0.4591836734693877, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4591836734693877 ...........
[CV]  criterion=gini, min_impurity_split=0.4591836734693877, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4693877551020408 ...........
[CV]  criterion=gini, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4693877551020408 ...........
[CV]  criterion=gini, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4693877551020408 ...........
[CV]  criterion=gini, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4693877551020408 ...........
[CV]  criterion=gini, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4693877551020408 ...........
[CV]  criterion=gini, min_impurity_split=0.4693877551020408, total=   0.0s
[CV] criterion=gini, min_impurity_split=0.4795918

[CV] .............................. min_samples_split=8, total=   0.0s
[CV] min_samples_split=8 .............................................
[CV] .............................. min_samples_split=8, total=   0.0s
[CV] min_samples_split=8 .............................................
[CV] .............................. min_samples_split=8, total=   0.0s
[CV] min_samples_split=8 .............................................
[CV] .............................. min_samples_split=8, total=   0.0s
[CV] min_samples_split=8 .............................................
[CV] .............................. min_samples_split=8, total=   0.0s
[CV] min_samples_split=10 ............................................
[CV] ............................. min_samples_split=10, total=   0.0s
[CV] min_samples_split=10 ............................................
[CV] ............................. min_samples_split=10, total=   0.0s
[CV] min_samples_split=10 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done 610 out of 610 | elapsed:    3.4s finished
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'criterion': ['entropy'], 'min_impurity_split': array([0.     , 0.02041, 0.04082, 0.06122, 0.08163, 0.10204, 0.12245,
       0.14286, 0.16327, 0.18367, 0.20408, 0.22449, 0.2449 , 0.26531,
       0.28571, 0.30612, 0.32653, 0.34694, 0.36735, 0.38776, 0.40816,
       0.42857, 0.44898, 0.46...59, 0.4898 ,
       0.5    ])}, {'max_depth': range(2, 10)}, {'min_samples_split': range(2, 30, 2)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [ ]:
clf.best_score_

In [198]:
clf.best_params_

{'criterion': 'entropy', 'min_impurity_split': 0.673469387755102}

In [199]:
clf.best_estimator_.score(X_test, y_test)

0.8100558659217877

### 随机森林

In [200]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=500)


parameters = {'max_depth': list(range(2, 10)),
              'min_samples_split': list(range(2, 10)),
              'min_samples_leaf': list(range(2, 10)),
              'max_features': list(range(1, 9))}

clf = GridSearchCV(rfr, parameters, cv=5,verbose=2)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 4096 candidates, totalling 20480 fits
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3 
[CV]  max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3, total=   0.3s
[CV] max_depth=2, max_features=1, min_samples_leaf=2, min_samples_split=3 
[CV]  max_

...

[CV]  max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2 
[CV]  max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2 
[CV]  max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2 
[CV]  max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=2, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=3 
[CV]  max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=3, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=3 
[CV]  max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=3, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=3, min_samples_split=3 
[CV]  max_

[CV]  max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4 
[CV]  max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4, total=   0.4s
[CV] max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4 
[CV]  max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4 
[CV]  max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=4, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=5 
[CV]  max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=5, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=5 
[CV]  max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=5, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=4, min_samples_split=5 
[CV]  max_

[CV]  max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6 
[CV]  max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6 
[CV]  max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6 
[CV]  max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=6, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=7 
[CV]  max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=7, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=7 
[CV]  max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=7, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=5, min_samples_split=7 
[CV]  max_

[CV]  max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8 
[CV]  max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8 
[CV]  max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8 
[CV]  max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=8, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=9 
[CV]  max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=9, total=   0.3s
[CV] max_depth=2, max_features=4, min_samples_leaf=6, min_samples_split=9 


KeyboardInterrupt: 

In [133]:
clf.best_params_

{'max_depth': 4, 'n_estimators': 20}

In [134]:
clf.best_score_

0.4424730367805543

In [128]:
entropy_thresholds

array([0.        , 0.02040816, 0.04081633, 0.06122449, 0.08163265,
       0.10204082, 0.12244898, 0.14285714, 0.16326531, 0.18367347,
       0.20408163, 0.2244898 , 0.24489796, 0.26530612, 0.28571429,
       0.30612245, 0.32653061, 0.34693878, 0.36734694, 0.3877551 ,
       0.40816327, 0.42857143, 0.44897959, 0.46938776, 0.48979592,
       0.51020408, 0.53061224, 0.55102041, 0.57142857, 0.59183673,
       0.6122449 , 0.63265306, 0.65306122, 0.67346939, 0.69387755,
       0.71428571, 0.73469388, 0.75510204, 0.7755102 , 0.79591837,
       0.81632653, 0.83673469, 0.85714286, 0.87755102, 0.89795918,
       0.91836735, 0.93877551, 0.95918367, 0.97959184, 1.        ])

In [129]:
clf.best_estimator_.score(X_test, y_test)

0.7877094972067039